In [ ]:
import cv2
import os
import numpy as np

def preprocess_images(input_dir, output_dir, clip_limit=2.0, tile_grid_size=(8, 8), sigma=0.33, kernel_size=5):
    """
    Preprocess images using CLAHE and an improved Canny edge detection method for brain MRI tumor detection.

    Args:
        input_dir (str): Path to the input directory containing images.
        output_dir (str): Path to the output directory for processed images.
        clip_limit (float): Contrast limiting for CLAHE.
        tile_grid_size (tuple): Size of the tiles for CLAHE.
        sigma (float): Factor for calculating adaptive thresholds in Canny.
        kernel_size (int): Size of Gaussian blur kernel to smooth the image.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    for root, _, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)

        for img_name in files:
            try:
                img_path = os.path.join(root, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Skipped: {img_path} (not a valid image)")
                    continue

                # Apply CLAHE for contrast enhancement
                img_clahe = clahe.apply(img)

                # Apply Gaussian Blur to reduce noise
                img_blur = cv2.GaussianBlur(img_clahe, (kernel_size, kernel_size), 0)

                # Compute median and derive adaptive thresholds
                v = int(np.median(img_blur))
                lower = max(0, int((1.0 - sigma) * v))
                upper = min(255, int((1.0 + sigma) * v))

                # Apply Canny edge detection
                img_canny = cv2.Canny(img_blur, lower, upper)

                # Apply morphological closing to remove small gaps and noise
                kernel = np.ones((3, 3), np.uint8)
                img_canny = cv2.morphologyEx(img_canny, cv2.MORPH_CLOSE, kernel)

                # Save the processed image
                output_path = os.path.join(output_subdir, img_name)
                cv2.imwrite(output_path, img_canny)
            except Exception as e:
                print(f"Error processing {img_name}: {e}")

    print(f"Processing complete. Processed images saved in: {output_dir}")

# Example usage:
preprocess_images("data/raw_images", "data/processed_image", clip_limit=2.0, tile_grid_size=(8, 8), sigma=0.95, kernel_size=5)


Processing complete. Processed images saved in: data/processed_image


In [5]:
import os
import cv2
import numpy as np
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

# Dataset class for multiple directories
class MultiFolderBrainTumorDataset(Dataset):
    def __init__(self, image_dirs, labels, transform=None):
        self.image_dirs = image_dirs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = None
        for image_dir in self.image_dirs:
            potential_path = os.path.join(image_dir, self.labels[idx]['image'])
            if os.path.exists(potential_path):
                img_path = potential_path
                break

        if img_path is None:
            raise FileNotFoundError(f"Image {self.labels[idx]['image']} not found in directories {self.image_dirs}")

        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label

# Function to generate labels from multiple directories
def generate_labels_multiple_dirs(image_dirs):
    labels = []
    for label, folder in enumerate(['yes', 'no']):
        for image_dir in image_dirs:
            folder_path = os.path.join(image_dir, folder)
            if not os.path.exists(folder_path):
                continue
            for img_name in os.listdir(folder_path):
                labels.append({'image': os.path.join(folder, img_name), 'label': label})
    return labels

# Grad-CAM class
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def generate_heatmap(self, gradients, activations):
        pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
        activations = activations[0]  # Batch size is 1
        for i in range(activations.shape[0]):
            activations[i, :, :] *= pooled_gradients[i]

        heatmap = torch.mean(activations, dim=0).cpu().detach().numpy()
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
        return heatmap

    def __call__(self, input_tensor, class_idx=None):
        self.model.eval()
        with torch.enable_grad():  # Ensure gradients are enabled
            output = self.model(input_tensor)

            if class_idx is None:
                class_idx = torch.argmax(output, dim=1).item()

            self.model.zero_grad()
            class_score = output[:, class_idx]
            class_score.backward(retain_graph=True)

            heatmap = self.generate_heatmap(self.gradients, self.activations)
        return heatmap


# Save Grad-CAM images
def save_gradcam_images(gradcam, input_tensor, original_image_paths, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i in range(input_tensor.size(0)):
        heatmap = gradcam(input_tensor[i:i + 1])

        # Load original image for overlay
        original_image = cv2.imread(original_image_paths[i])
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        # Resize heatmap to original image size
        heatmap_resized = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))
        heatmap_resized = np.uint8(255 * heatmap_resized)
        heatmap_colored = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)

        # Overlay heatmap on the original image
        overlayed_image = cv2.addWeighted(original_image, 0.6, heatmap_colored, 0.4, 0)

        # Save overlayed image
        output_path = os.path.join(output_folder, f"gradcam_{i}.jpg")
        cv2.imwrite(output_path, cv2.cvtColor(overlayed_image, cv2.COLOR_RGB2BGR))

# Dataset setup
processed_image_dir = 'data/processed_image'
raw_image_dir = 'data/raw_images'
image_dirs = [processed_image_dir, raw_image_dir]

# Generate labels and split dataset
labels = generate_labels_multiple_dirs(image_dirs)
train_labels, val_labels = train_test_split(labels, test_size=0.2, random_state=42)

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and dataloaders
train_dataset = MultiFolderBrainTumorDataset(image_dirs, train_labels, transform)
val_dataset = MultiFolderBrainTumorDataset(image_dirs, val_labels, transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Model setup
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes: tumor (1) and no tumor (0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

# Grad-CAM setup
gradcam = GradCAM(model, model.layer4)
gradcam_output_dir = "gradcam_output"

# Training loop with Grad-CAM during validation
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward and backward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/10], Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (images, labels) in enumerate(val_dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Save Grad-CAM images
            image_paths = [os.path.join(image_dirs[0], label['image']) for label in val_labels[idx * 64:(idx + 1) * 64]]
            save_gradcam_images(gradcam, images, image_paths, gradcam_output_dir)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")


cuda
Epoch [1/10], Loss: 0.5012
Validation Loss: 0.3603, Accuracy: 85.34%
Epoch [2/10], Loss: 0.2742
Validation Loss: 0.2474, Accuracy: 90.09%
Epoch [3/10], Loss: 0.1750
Validation Loss: 0.1656, Accuracy: 94.18%
Epoch [4/10], Loss: 0.1117
Validation Loss: 0.1090, Accuracy: 98.17%
Epoch [5/10], Loss: 0.0614
Validation Loss: 0.0719, Accuracy: 98.92%
Epoch [6/10], Loss: 0.0358
Validation Loss: 0.0528, Accuracy: 99.35%
Epoch [7/10], Loss: 0.0241
Validation Loss: 0.0402, Accuracy: 99.35%
Epoch [8/10], Loss: 0.0174
Validation Loss: 0.0327, Accuracy: 99.57%
Epoch [9/10], Loss: 0.0116
Validation Loss: 0.0305, Accuracy: 99.35%
Epoch [10/10], Loss: 0.0090
Validation Loss: 0.0284, Accuracy: 99.35%


In [6]:
# Save the model
torch.save(model.state_dict(), 'brain_tumor_model_processed.pth')
print("Model saved as 'brain_tumor_model_processed.pth'.")

# Inference with a sample image
def predict_sample(image_path, model, transform, device):
    model.eval()  # Set model to evaluation mode
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# Load the trained model for inference
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # Make sure model is initialized
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust the output layer if needed
model.load_state_dict(torch.load('brain_tumor_model_processed.pth'))  # Load weights
model = model.to(device)  # Move the model to the correct device

# 2 classes: tumor (1) and no tumor (0)
classes = ['Tumor', 'No Tumor']

# Path to the 'sample' folder containing images
sample_folder_path = 'sample'  # Update with the correct path to your folder

# Get all image files from the folder
image_files = [f for f in os.listdir(sample_folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

# Iterate over each image file in the folder and make predictions
for image_file in image_files:
    image_path = os.path.join(sample_folder_path, image_file)
    prediction = predict_sample(image_path, model, transform, device)
    print(f"Prediction for {image_file}: {classes[prediction]}")


Model saved as 'brain_tumor_model_processed.pth'.
Prediction for 47 no.jpg: No Tumor
Prediction for 48 no.jpeg: No Tumor
Prediction for 49 no.jpg: No Tumor
Prediction for 50 no.jpg: No Tumor
Prediction for no 100.jpg: No Tumor
Prediction for no 99.jpg: No Tumor
Prediction for no.jpg: No Tumor
Prediction for No18.jpg: No Tumor
Prediction for No19.jpg: No Tumor
Prediction for No20.jpg: No Tumor
Prediction for Y164.JPG: No Tumor
Prediction for Y165.JPG: No Tumor
Prediction for Y169.jpg: Tumor
Prediction for Y183.jpg: No Tumor
Prediction for Y192.JPG: Tumor
Prediction for Y243.JPG: No Tumor
Prediction for Y244.JPG: Tumor
Prediction for Y246.JPG: No Tumor
Prediction for Y255.JPG: Tumor
Prediction for Y258.JPG: No Tumor
